<a href="https://colab.research.google.com/github/nathUjjal/Multi-Modal-Fake-News-Detection/blob/main/notebooks/imageBasedDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 22.6 MB/s eta 0:00:00


In [5]:
from transformers import pipeline,BlipProcessor,BlipForConditionalGeneration
import torch
from io import BytesIO
import requests
from PIL import Image
import numpy as np
import easyocr

In [2]:
model=BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
processor=BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [9]:
classifier=pipeline('text-classification',model="facebook/bart-large-mnli")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [7]:
def generate_caption(image):
    inputs=processor(image,return_tensors="pt")
    out=model.generate(**inputs)
    caption=processor.decode(out[0],skip_special_tokens=True)
    return caption

In [6]:
def extract_test(image):
  ocr=easyocr.Reader(['en'])
  extracted_text=ocr.readtext(image,details=0)
  return extracted_text

In [11]:
def retrieve_evidence(claim,max_sentence=2):
  pass

In [10]:
def verify_claim(claim,evidence):
  results=classifier({'text':evidence,'text_pair':claim},truncation=True)
  label=results[0].label
  score=results[0].score
  return label,score

In [13]:
def image_based_verdict(label,score):
    if label=='contradiction' and score > 0.7 :
      return {'img_status':"FAKE",'img_score':score}
    elif label=='entailment' and score > 0.7 :
      return {'img_status':"REAL",'img_score':score}
    else:
      return {'img_status':"UNCERTAIN",'img_score':score}